# Ingest ISO 3166 data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [1]:
import os
import numpy as np
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc
# import python_pachyderm

Define Environment and Execution Variables

In [ ]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

In [4]:
import osc_ingest_trino as osc

trino_bucket = osc.attach_s3_bucket("S3_DEV")

Open a Trino connection using JWT for authentication

In [5]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
essd_schema ='sandbox'
essd_prefix = 'essd_'

In [6]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('pcaf_sovereign_footprint',)
('sandbox',)


The python module `pycountry` has all the data we need, and we can use it within python/pandas.

We import the data here so we can use it in SQL/Trino queries.

In [7]:
import pycountry

Enter the Pandas!

In [8]:
import pandas as pd
import numpy as np
import io

Drop previous tables and schema to start with a fresh slate

In [9]:
for ingest_table in [ 'countries', 'subdivisions' ]:
    sql = f"""
drop table if exists {ingest_schema}.{ingest_table}
"""
    print(sql)
    qres = engine.execute(sql)
    print(qres.fetchall())


drop table if exists sandbox.countries

[(True,)]

drop table if exists sandbox.subdivisions

[(True,)]


In [10]:
sql = f"show tables in {ingest_schema}"
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

show tables in sandbox
[('assets_earnings_investments',), ('average_fx',), ('census_all_sector_survey_2017',), ('census_sic4',), ('census_sic_ranges',), ('closing_fx',), ('customers_sales',), ('debt_equity_returns',), ('dera_num',), ('dera_sub',), ('dera_tag',), ('emissions_targets',), ('employees',), ('epa_co2_injection',), ('epa_direct_emitters',), ('epa_gathering_boosting',), ('epa_geologic_sequestration_of_co2',), ('epa_ldc_direct_emissions',), ('epa_onshore_oil_gas_prod',), ('epa_parent_attribution',), ('epa_parent_company',), ('epa_sankey',), ('epa_sf6_from_elec_equip',), ('epa_suppliers',), ('epa_transmission_pipelines',), ('essd_ch4_gwp',), ('essd_ghg_data',), ('essd_gwp100_data',), ('essd_gwp_100yr',), ('essd_lulucf',), ('essd_regions',), ('essd_sectors',), ('expenditure_bills_burden',), ('financials_by_lei',), ('gleif_direct_issuer_ultimate_issuer',), ('gleif_isin_lei',), ('gppd',), ('housing_units_income',), ('isic_to_sector',), ('naics_isic_sic',), ('naics_sic_crosswalk',),

One table we want to link up with is defined in the ESSD schema.  It links ISO Alpha3 codes with UN regions.

In [11]:
engine.execute(f"select * from {essd_schema}.{essd_prefix}regions").fetchall()[0:5]

[('AFG', 'Afghanistan', 'Asia and developing Pacific', 'Southern Asia', 'Rest of Southern Asia', 'ldc'),
 ('ALA', 'Åland Islands', 'Developed Countries', 'Europe', 'Northern and western Europe', 'developed'),
 ('ALB', 'Albania', 'Developed Countries', 'Europe', 'Southern and eastern Europe', 'developed'),
 ('DZA', 'Algeria', 'Africa', 'Africa', 'North Africa', 'developing'),
 ('ASM', 'American Samoa', 'Asia and developing Pacific', 'South-East Asia and developing Pacific', 'Developing Pacific', 'developing')]

In [12]:
df = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df.replace({np.NaN:pd.NaT}, inplace=True)
df_countries = df.convert_dtypes()
df_countries

,alpha_2,alpha_3,name,numeric,official_name,common_name
0,AW,ABW,Aruba,533,<NA>,<NA>
1,AF,AFG,Afghanistan,004,Islamic Republic of Afghanistan,<NA>
2,AO,AGO,Angola,024,Republic of Angola,<NA>
3,AI,AIA,Anguilla,660,<NA>,<NA>
4,AX,ALA,Åland Islands,248,<NA>,<NA>
...,...,...,...,...,...,...
244,WS,WSM,Samoa,882,Independent State of Samoa,<NA>
245,YE,YEM,Yemen,887,Republic of Yemen,<NA>
246,ZA,ZAF,South Africa,710,Republic of South Africa,<NA>
247,ZM,ZMB,Zambia,894,Republic of Zambia,<NA>


In [13]:
df = pd.DataFrame([subdivision.__dict__['_fields'] for subdivision in pycountry.subdivisions])
display(df[df.parent_code.notna()])
df.replace({np.NaN:pd.NaT}, inplace=True)
df_subdivisions = df.convert_dtypes()
df_subdivisions

,code,name,type,parent_code,country_code,parent
68,AL-BR,Berat,District,AL-01,AL,01
69,AL-BU,Bulqizë,District,AL-09,AL,09
70,AL-DI,Dibër,District,AL-09,AL,09
71,AL-DL,Delvinë,District,AL-12,AL,12
72,AL-DR,Durrës,District,AL-02,AL,02
...,...,...,...,...,...,...
4615,UG-415,Kyenjojo,District,UG-W,UG,W
4616,UG-416,Ibanda,District,UG-W,UG,W
4617,UG-417,Isingiro,District,UG-W,UG,W
4618,UG-418,Kiruhura,District,UG-W,UG,W


,code,name,type,parent_code,country_code,parent
0,AD-02,Canillo,Parish,<NA>,AD,<NA>
1,AD-03,Encamp,Parish,<NA>,AD,<NA>
2,AD-04,La Massana,Parish,<NA>,AD,<NA>
3,AD-05,Ordino,Parish,<NA>,AD,<NA>
4,AD-06,Sant Julià de Lòria,Parish,<NA>,AD,<NA>
...,...,...,...,...,...,...
4878,ZW-MI,Midlands,Province,<NA>,ZW,<NA>
4879,ZW-MN,Matabeleland North,Province,<NA>,ZW,<NA>
4880,ZW-MS,Matabeleland South,Province,<NA>,ZW,<NA>
4881,ZW-MV,Masvingo,Province,<NA>,ZW,<NA>


Another table we want to link with are the OECD countries (scraped off their public web page): https://www.oecd.org/about/document/ratification-oecd-convention.htm

In [14]:
oecd_to_iso_dict = {'Czech Republic':'Czechia',
                    'Korea':'Korea, Republic of',
                    'Slovak Republic':'Slovakia'}

oecd_df = pd.read_csv('../data/external/oecd-countries-20220605.csv',
                      parse_dates=['Date']).convert_dtypes()
osc.enforce_sql_column_names(oecd_df, inplace=True)
oecd_df.rename(columns={'country':'name', 'date':'oecd_date'}, inplace=True)
oecd_df.name = oecd_df.name.str.title().replace(oecd_to_iso_dict)          

In [15]:
assert(oecd_df.merge(df_countries, how='left', on='name').name.notna().all())

In [16]:
df_countries = df_countries.merge(oecd_df, how='left', on='name')

In [17]:
for ingest_table, df in [ ('countries',df_countries), ('subdivisions', df_subdivisions)]:
    table_check = engine.execute(f'drop table if exists {ingest_schema}.{ingest_table}')
    for row in table_check.fetchall():
        print(row)
    columnschema = osc.create_table_schema_pairs(df, typemap={'datetime64[ns]':'timestamp(6)'})
    tabledef = f"""
create table if not exists {ingest_schema}.{ingest_table} (
{columnschema}
) with (
format = 'ORC'
)
"""
    print(tabledef)

    table_create = engine.execute(tabledef)
    for row in table_create.fetchall():
        print(row)
    df.to_sql(ingest_table,
              con=engine, schema=ingest_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))
    dataset_query = (f'SELECT * FROM {ingest_schema}.{ingest_table} limit 10')
    print(dataset_query)
    dataset = engine.execute(dataset_query)
    for row in dataset.fetchall():
        print(row)

(True,)

create table if not exists sandbox.countries (
    alpha_2 varchar,
    alpha_3 varchar,
    name varchar,
    numeric varchar,
    official_name varchar,
    common_name varchar,
    oecd_date timestamp(6)
) with (
format = 'ORC'
)

(True,)
SELECT * FROM sandbox.countries limit 10
('AW', 'ABW', 'Aruba', '533', None, None, None)
('AF', 'AFG', 'Afghanistan', '004', 'Islamic Republic of Afghanistan', None, None)
('AO', 'AGO', 'Angola', '024', 'Republic of Angola', None, None)
('AI', 'AIA', 'Anguilla', '660', None, None, None)
('AX', 'ALA', 'Åland Islands', '248', None, None, None)
('AL', 'ALB', 'Albania', '008', 'Republic of Albania', None, None)
('AD', 'AND', 'Andorra', '020', 'Principality of Andorra', None, None)
('AE', 'ARE', 'United Arab Emirates', '784', None, None, None)
('AR', 'ARG', 'Argentina', '032', 'Argentine Republic', None, None)
('AM', 'ARM', 'Armenia', '051', 'Republic of Armenia', None, None)
(True,)

create table if not exists sandbox.subdivisions (
    code v